# Задание

1) Датасет ml-latest.

2) Вспомнить подходы, которые мы разбирали.

3) Выбрать понравившийся подход к гибридным системам.

4) Написать свою. [Материалы вы найдёте здесь.](https://github.com/ALKONDR/netology-recsys/blob/master/lecture-5/lecture-5-part-2.ipynb)

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162669 sha256=fcf2e9579ded788c9f6f8920f09d75ceb34bf0382d914acea1b53bdb9e010837
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, NMF
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808330 sha256=e23a88c42181cc998d2e60517ac08e64c40977757d2e2109b68615b6f9c5c445
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
from lightfm import LightFM

In [ ]:
from surprise import Dataset

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-02-09 12:38:10--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 142.251.163.100, 142.251.163.101, 142.251.163.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.163.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-02-09 12:38:10--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.167.132, 2607:f8b0:4004:c1d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.167.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.05s   

2024-02-09 12:38:11 (17.7 M

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

#Первый алгоритм (SVD)

In [ ]:
# Инициализация и обучение модели SVD
algo = SVD()
algo.fit(trainset)

In [ ]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8739


0.8739461515551583

In [ ]:
algo.predict(uid=2.0, iid='Mortal Kombat (1995')

Prediction(uid=2.0, iid='Mortal Kombat (1995', r_ui=None, est=3.5618447974150387, details={'was_impossible': False})

In [ ]:
current_user_id = 2.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue

    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [ ]:
best_indices = np.argsort(scores)[-10:]
for i in reversed(best_indices):
    print(titles[i], scores[i])

Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 4.527578642972248
Lawrence of Arabia (1962) 4.462491502701079
Casablanca (1942) 4.4486648328295715
Memento (2000) 4.442960756165323
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) 4.432645098541968
Philadelphia Story, The (1940) 4.414995865951734
Patton (1970) 4.410847445397311
Fish Called Wanda, A (1988) 4.410765380163006
Star Wars: Episode V - The Empire Strikes Back (1980) 4.405539121044134
Third Man, The (1949) 4.403833783821812


#Второй алоритм (NMF)

In [ ]:
# Инициализация и обучение модели NMF
algo2 = NMF(n_epochs=80, n_factors=120)
algo2.fit(trainset)

In [ ]:
test_pred2 = algo2.test(testset)
accuracy.rmse(test_pred2, verbose=True)

RMSE: 0.8832


0.883163287330031

In [ ]:
algo2.predict(uid=2.0, iid='Mortal Kombat (1995')

Prediction(uid=2.0, iid='Mortal Kombat (1995', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and item are unknown.'})

In [ ]:
current_user_id = 2.0
user_moviesNMF = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scoresNMF = []
titlesNMF = []

for movie in movies_with_ratings.title.unique():
    if movie in user_moviesNMF:
        continue

    scoresNMF.append(algo2.predict(uid=current_user_id, iid=movie).est)
    titlesNMF.append(movie)

In [ ]:
best_indicesNMF = np.argsort(scoresNMF)[-10:]
for i in reversed(best_indicesNMF):
    print(titlesNMF[i], scoresNMF[i])

Separation, A (Jodaeiye Nader az Simin) (2011) 4.820930770970769
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) 4.677903795515848
Black Mirror: White Christmas (2014) 4.6562531620843375
Grave of the Fireflies (Hotaru no haka) (1988) 4.639529498071562
General, The (1926) 4.63288385153855
Outlaw Josey Wales, The (1976) 4.617410868620786
Last Detail, The (1973) 4.611963255779537
Last Tango in Paris (Ultimo tango a Parigi) (1972) 4.604310090739301
Touch of Evil (1958) 4.601821998752564
Hour of the Wolf (Vargtimmen) (1968) 4.598541093761971


#Соединяем два алгоритма

In [ ]:
def recommend_for_user(user_id):
    user_movies = movies_with_ratings[movies_with_ratings.userId == user_id].title.unique()

    # первый
    scores = []
    titles = []

    for movie in movies_with_ratings.title.unique():
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)

    best_indices = np.argsort(scores)[-10:]
    for i in reversed(best_indices):
        print(titles[i], scores[i])

    # второй
    scoresNMF = []
    titlesNMF = []

    for movie in movies_with_ratings.title.unique():
        if movie in user_moviesNMF:
            continue

        scoresNMF.append(algo2.predict(uid=current_user_id, iid=movie).est)
        titlesNMF.append(movie)

    best_indicesNMF = np.argsort(scoresNMF)[-10:]
    for i in reversed(best_indicesNMF):
        print(titlesNMF[i], scoresNMF[i])

In [ ]:
movies_with_ratings[movies_with_ratings.userId == 2.0].sort_values('rating').tail()

,movieId,title,genres,userId,rating,timestamp
91873,80906,Inside Job (2010),Documentary,2,5.0,1445715172
87775,60756,Step Brothers (2008),Comedy,2,5.0,1445714980
93494,89774,Warrior (2011),Drama,2,5.0,1445715189
96369,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama,2,5.0,1445714966
98703,131724,The Jinx: The Life and Deaths of Robert Durst ...,Documentary,2,5.0,1445714851


In [ ]:
recommend_for_user(2)

Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 4.527578642972248
Lawrence of Arabia (1962) 4.462491502701079
Casablanca (1942) 4.4486648328295715
Memento (2000) 4.442960756165323
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) 4.432645098541968
Philadelphia Story, The (1940) 4.414995865951734
Patton (1970) 4.410847445397311
Fish Called Wanda, A (1988) 4.410765380163006
Star Wars: Episode V - The Empire Strikes Back (1980) 4.405539121044134
Third Man, The (1949) 4.403833783821812
Separation, A (Jodaeiye Nader az Simin) (2011) 4.820930770970769
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) 4.677903795515848
Black Mirror: White Christmas (2014) 4.6562531620843375
Grave of the Fireflies (Hotaru no haka) (1988) 4.639529498071562
General, The (1926) 4.63288385153855
Outlaw Josey Wales, The (1976) 4.617410868620786
Last Detail, The (1973) 4.611963255779537
Last Tango in Paris (Ultimo tango a Parigi) (1972) 4.604310090739301
Touch of Evil (1958) 4.601821998752